<a href="https://colab.research.google.com/github/jenova13q/paw_finder/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_clip_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%%capture
!pip install torch torchvision
!pip install git+https://github.com/openai/CLIP.git
!pip install pillow
!pip install numpy

In [6]:
%%capture
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [7]:
import torch
import clip
from PIL import Image
import os
from tqdm import tqdm
import random
import numpy as np
import matplotlib.pyplot as plt

import json
import os
import random
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageEnhance, ImageOps
import clip
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image, ImageEnhance, ImageOps

from transformers import CLIPProcessor, CLIPModel
from torch.optim import AdamW
from torch.optim import lr_scheduler
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn

from sklearn.model_selection import train_test_split

In [9]:
images_path = "/content/drive/MyDrive/Colab Notebooks/parses/cat_images"

In [8]:
import json

# Функция для создания описания
def create_description(pet_id, details):
    description = ""#f"**Cat ID:** {pet_id}\n"

    age = details.get('age')
    primary_breed = details.get('primary_breed')
    species = details.get('species', 'Кошка')
    color = details.get('color')
    hair_length = details.get('hair_length')
    sex = details.get('sex')
    purebred = details.get('purebred')

    if age:
        description += f"{age} "
    if primary_breed:
        description += f"{primary_breed} "
    else:
        description += f"{species.lower()} "
    if color:
        description += f"cat with {color.lower()} color. This is a "
    if purebred is not None:
        description += f"{'purebred ' if purebred else ''}"
    if sex:
        sex_string = "female cat " if sex == 'Female' else "male cat "
        description += sex_string
    if hair_length:
        description += f"with {hair_length.lower()} hair."

    return description.strip() + "\n"

# Чтение данных из файла
with open('/content/drive/MyDrive/Colab Notebooks/parses/detailed_pets_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Создание описаний для каждого питомца
descriptions = {}
for pet_id, pet_data in data.items():
    formatted_pet_id = str(pet_id).zfill(6)
    details = pet_data.get('details', {})
    description = create_description(formatted_pet_id, details)
    descriptions[formatted_pet_id] = description

In [ ]:
print(descriptions['000001'])

Young Siamese cat with white (mostly) color. This is a purebred female cat with short hair.



In [ ]:
print(descriptions['000002'])

Young Tabby cat with gray or blue color. This is a female cat with medium hair.



In [10]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [11]:
set_seed()

Random seed set as 42


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [13]:
cat_images = [img for img in os.listdir(images_path)]
random.shuffle(cat_images)

# Создание словаря для преобразования меток
unique_labels = list(set([img.split('_')[0] for img in cat_images]))
random.shuffle(unique_labels)


label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

# Определение количества кошек для каждого набора
num_cats = len(unique_labels)
num_train = int(0.7 * num_cats)
num_val = int(0.15 * num_cats)
num_test = num_cats - num_train - num_val

# Разделение кошек на три группы
train_labels = unique_labels[:num_train]
val_labels = unique_labels[num_train:num_train + num_val]
test_labels = unique_labels[num_train + num_val:]

# Разделение изображений на три группы
train_images = [img for img in cat_images if img.split('_')[0] in train_labels]
val_images = [img for img in cat_images if img.split('_')[0] in val_labels]
test_images = [img for img in cat_images if img.split('_')[0] in test_labels]

# Проверка результатов разделения
print(f"Train images: {len(train_images)}")
print(f"Validation images: {len(val_images)}")
print(f"Test images: {len(test_images)}")
print(f"Number of train cats: {len(train_labels)}")
print(f"Number of validation cats: {len(val_labels)}")
print(f"Number of test cats: {len(test_labels)}")

Train images: 7167
Validation images: 1498
Test images: 1566
Number of train cats: 2672
Number of validation cats: 572
Number of test cats: 574


In [24]:
model, processor = clip.load("ViT-B/32", device=device, jit=False)
clip.model.convert_weights(model)

In [15]:
class CatDataset(Dataset):
    def __init__(self, images, label_to_idx, pet_details):
        self.images = images
        self.label_to_idx = label_to_idx
        self.pet_details = pet_details

        self.processed_cache = {}
        for img_path in self.images:
            self.processed_cache[img_path] = processor(Image.open(f"{images_path}/{img_path}"))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = self.processed_cache[img_path]
        label = img_path.split('_')[0]
        label_idx = self.label_to_idx[label]
        details_text = clip.tokenize(self.pet_details[label])

        return image, label_idx, details_text

In [21]:
batch_size = 32
epoch_num = 20

In [22]:
train_dataset = CatDataset(train_images,  label_to_idx, descriptions)
val_dataset = CatDataset(val_images,  label_to_idx, descriptions)
test_dataset = CatDataset(test_images,  label_to_idx, descriptions)

In [23]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [29]:
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

In [30]:
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader)*epoch_num)

In [31]:
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

In [32]:
def train_model(train_loader, val_loader, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Эпоха {epoch + 1}/{num_epochs} - Обучение")
        for images, labels, details_text in progress_bar:
            images = images.to(device)
            labels = labels.to(device)
            details_text = details_text.squeeze(1).to(device)

            optimizer.zero_grad()

            logits_per_image, logits_per_text = model(images, details_text)

            ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
            total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
            total_loss.backward()
            convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            clip.model.convert_weights(model)

            running_loss += total_loss.item()
            progress_bar.set_postfix(loss=running_loss / (progress_bar.n + 1))
        print(f"Эпоха {epoch + 1} завершена. Потеря на тренировочном наборе: {running_loss / len(train_loader)}")

        # Оценка на валидационном наборе
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels, details_text in tqdm(val_loader, desc=f"Эпоха {epoch + 1}/{num_epochs} - Валидация"):
                images = images.to(device)
                labels = labels.to(device)
                details_text = details_text.squeeze(1).to(device)

                logits_per_image, logits_per_text = model(images, details_text)

                ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
                total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2

                val_loss += total_loss.item()
            print(f"Потеря на валидационном наборе: {val_loss / len(val_loader)}")

# Обучение модели
train_model(train_loader, val_loader, optimizer, 20)


Эпоха 1/20 - Обучение: 100%|██████████| 224/224 [00:22<00:00,  9.84it/s, loss=2.08]


Эпоха 1 завершена. Потеря на тренировочном наборе: 2.0784170968191966


Эпоха 1/20 - Валидация: 100%|██████████| 47/47 [00:01<00:00, 27.95it/s]


Потеря на валидационном наборе: 2.0292137632978724


Эпоха 2/20 - Обучение: 100%|██████████| 224/224 [00:20<00:00, 10.76it/s, loss=1.43]


...

Потеря на валидационном наборе: 3.728266289893617


Эпоха 20/20 - Обучение: 100%|██████████| 224/224 [00:21<00:00, 10.53it/s, loss=0.113]


Эпоха 20 завершена. Потеря на тренировочном наборе: 0.11258073363985334


Эпоха 20/20 - Валидация: 100%|██████████| 47/47 [00:01<00:00, 29.67it/s]

Потеря на валидационном наборе: 3.7289311835106385


In [33]:
def save_model(model, optimizer, epoch, path="model_checkpoint.pth"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)

model_path = "/content/drive/MyDrive/Colab Notebooks/model_checkpoint_clip_new_1.pth"
save_model(model, optimizer, epoch=20, path=model_path)

In [ ]:
# Загрузка сохраненной модели
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/model_checkpoint_clip_new_1.pth"))
model.to(device)

In [34]:
train_features_list = []
train_labels = []

for image_name in tqdm(train_images, desc="Извлечение признаков для тренировочных данных"):
    image_path = os.path.join(images_path, image_name)
    image = Image.open(image_path)
    image = processor(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    train_features_list.append(image_features)
    train_labels.append(image_name.split('_')[0])

train_features_tensor = torch.cat(train_features_list, dim=0)

Извлечение признаков для тренировочных данных: 100%|██████████| 7167/7167 [01:48<00:00, 65.80it/s]


In [35]:
# Подготовка тестовых признаков и меток
test_features_list = []
test_labels = []
for image_name in test_images:
    image_path = os.path.join(images_path, image_name)
    image = Image.open(image_path)
    image = processor(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    test_features_list.append(image_features)
    test_labels.append(image_name.split('_')[0])

test_features_tensor = torch.cat(test_features_list, dim=0)



In [41]:
torch.save({
        'epoch': 20,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss_img,
        }, f"/content/drive/MyDrive/Colab Notebooks/model_checkpoint_clip_new_1.pt")

In [60]:
# Функция для оценки точности
def evaluate_accuracy(images, features_tensor, labels, model, preprocess, device, top_k_list):
    correct_counts = {k: 0 for k in top_k_list}
    total_count = len(test_images)

    for image_name in tqdm(images, desc="Оценка точности"):
        query_cat_id = image_name.split('_')[0]
        query_image_path = os.path.join(images_path, image_name)
        query_image = preprocess(Image.open(query_image_path)).unsqueeze(0).to(device)

        with torch.no_grad():
            query_image_features = model.encode_image(query_image)
            query_image_features /= query_image_features.norm(dim=-1, keepdim=True)

        # Исключение текущего изображения из features_tensor
        current_idx = images.index(image_name)
        current_feature = features_tensor[current_idx]
        other_features_tensor = torch.cat((features_tensor[:current_idx], features_tensor[current_idx + 1:]), dim=0)
        other_labels = labels[:current_idx] + labels[current_idx + 1:]

        similarities = (query_image_features @ other_features_tensor.T).cpu().numpy().flatten()
        sorted_indices = similarities.argsort()[::-1]

        for k in top_k_list:
            top_k_labels = [other_labels[idx] for idx in sorted_indices[:k]]
            if query_cat_id in top_k_labels:
                correct_counts[k] += 1

    accuracies = {k: correct_counts[k] / total_count for k in top_k_list}
    return accuracies



# Оценка точности
top_k_list = [1, 5, 10, 20, 100, 1000, 4000, 8000]
accuracies = evaluate_accuracy(test_images, test_features_tensor, test_labels, model, processor, device, top_k_list)

print("Точность предсказаний:")
for k, accuracy in accuracies.items():
    print(f"Top-{k}: {accuracy:.4f}")

Оценка точности: 100%|██████████| 1566/1566 [00:23<00:00, 65.53it/s]

Точность предсказаний:
Top-1: 0.3851
Top-5: 0.5658
Top-10: 0.6494
Top-20: 0.7248
Top-100: 0.8557
Top-1000: 0.8985
Top-4000: 0.9017
Top-8000: 0.9017


In [77]:
def evaluate_accuracy(images, features_tensor, labels, model, preprocess, device, top_k_list):
    correct_counts = {k: 0 for k in top_k_list}
    total_count = len(images)

    for image_name in tqdm(images, desc="Оценка точности"):
        query_cat_id = image_name.split('_')[0]
        query_image_path = os.path.join(images_path, image_name)
        query_image = preprocess(Image.open(query_image_path)).unsqueeze(0).to(device)

        with torch.no_grad():
            query_image_features = model.encode_image(query_image)
            query_image_features /= query_image_features.norm(dim=-1, keepdim=True)

        # Исключение текущего изображения из features_tensor
        current_idx = images.index(image_name)
        current_feature = features_tensor[current_idx].to(device)
        other_features_tensor = torch.cat((features_tensor[:current_idx], features_tensor[current_idx + 1:]), dim=0).to(device)
        other_labels = labels[:current_idx] + labels[current_idx + 1:]

        similarities = (query_image_features @ other_features_tensor.T).cpu().numpy().flatten()
        sorted_indices = similarities.argsort()[::-1]

        for k in top_k_list:
            top_k_labels = [other_labels[idx] for idx in sorted_indices[:k]]
            if query_cat_id in top_k_labels:
                correct_counts[k] += 1

    accuracies = {k: correct_counts[k] / total_count for k in top_k_list}
    return accuracies

# Подготовка тестовых признаков и меток
def prepare_features_and_labels(images, model, preprocess, device):
    features_list = []
    labels = []

    for image_name in tqdm(images, desc="Извлечение признаков"):
        image_path = os.path.join(images_path, image_name)
        image = Image.open(image_path).convert("RGB")
        image = preprocess(image).unsqueeze(0).to(device)

        with torch.no_grad():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)

        features_list.append(image_features.cpu())
        labels.append(image_name.split('_')[0])

    features_tensor = torch.cat(features_list, dim=0)
    return features_tensor, labels

# Подготовка тренировочных признаков и меток
train_features_tensor, train_labels = prepare_features_and_labels(train_images, model, processor, device)

# Оценка точности на тренировочном наборе
top_k_list = [1, 5, 10, 20, 100, 1000, 4000, 7167, 8000]
train_accuracies = evaluate_accuracy(train_images, train_features_tensor, train_labels, model, processor, device, top_k_list)

# Train images: 7167
# Number of train cats: 2672

print("Точность предсказаний на тренировочном наборе:")
for k, accuracy in train_accuracies.items():
    print(f"Top-{k}: {accuracy:.4f}")

Оценка точности: 100%|██████████| 7167/7167 [02:25<00:00, 49.22it/s]

Точность предсказаний на тренировочном наборе:
Top-1: 0.4710
Top-5: 0.6734
Top-10: 0.7449
Top-20: 0.8001
Top-100: 0.8776
Top-1000: 0.8942
Top-4000: 0.8961
Top-7167: 0.8962
Top-8000: 0.8962


In [78]:
# Test images: 1566
# Number of test cats: 574

top_k_list = [1, 5, 10, 20, 100, 1000, 1566, 1567]

test_accuracies = evaluate_accuracy(test_images, test_features_tensor, test_labels, model, processor, device, top_k_list)




print("Точность предсказаний на теством наборе:")
for k, accuracy in test_accuracies.items():
    print(f"Top-{k}: {accuracy:.4f}")

Оценка точности: 100%|██████████| 1566/1566 [00:24<00:00, 64.94it/s]

Точность предсказаний на теством наборе:
Top-1: 0.3851
Top-5: 0.5658
Top-10: 0.6494
Top-20: 0.7248
Top-100: 0.8557
Top-1000: 0.8985
Top-4000: 0.9017
Top-7167: 0.9017
Top-8000: 0.9017


In [79]:
model_, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
checkpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/model_checkpoint_clip_new_1.pt")

# Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"]
checkpoint['model_state_dict']["input_resolution"] = model_.input_resolution #default is 224
checkpoint['model_state_dict']["context_length"] = model_.context_length # default is 77
checkpoint['model_state_dict']["vocab_size"] = model_.vocab_size

model_.load_state_dict(checkpoint['model_state_dict'])

AttributeError: 'CLIP' object has no attribute 'input_resolution'

In [80]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка модели и процессора
model_, preprocess = clip.load("ViT-B/32", device=device, jit=False) # Must set jit=False for training

# Загрузка контрольной точки (checkpoint)
checkpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/model_checkpoint_clip_new_1.pt", map_location=device)

# Загрузка состояния модели
model_.load_state_dict(checkpoint['model_state_dict'])

print("Модель успешно загружена")

Модель успешно загружена


In [81]:
test_accuracies = evaluate_accuracy(test_images, test_features_tensor, test_labels, model_, preprocess, device, top_k_list)




Оценка точности: 100%|██████████| 1566/1566 [00:24<00:00, 63.65it/s]


In [ ]:
# Настройка последних слоев для дообучения
for param in model.parameters():
    param.requires_grad = False

# Настройка некоторых параметров для дообучения
model.visual.proj.requires_grad = True
model.transformer.resblocks[-1].mlp.c_fc.requires_grad = True
model.transformer.resblocks[-1].mlp.c_proj.requires_grad = True

# Определение функции потерь и оптимизатора
loss_img = torch.nn.CrossEntropyLoss()
loss_txt = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader)*epoch_num)

# Функция для обучения модели
def train_model_(train_loader, val_loader, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Эпоха {epoch + 1}/{num_epochs} - Обучение")
        for images, labels, details_text in progress_bar:
            images = images.to(device)
            labels = labels.to(device)
            details_text = details_text.squeeze(1).to(device)

            optimizer.zero_grad()

            logits_per_image, logits_per_text = model(images, details_text)

            ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
            total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
            total_loss.backward()
            convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            clip.model.convert_weights(model)

            running_loss += total_loss.item()
            progress_bar.set_postfix(loss=running_loss / (progress_bar.n + 1))
        print(f"Эпоха {epoch + 1} завершена. Потеря на тренировочном наборе: {running_loss / len(train_loader)}")

        # Оценка на валидационном наборе
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels, details_text in tqdm(val_loader, desc=f"Эпоха {epoch + 1}/{num_epochs} - Валидация"):
                images = images.to(device)
                labels = labels.to(device)
                details_text = details_text.squeeze(1).to(device)

                logits_per_image, logits_per_text = model(images, details_text)

                ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
                total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2

                val_loss += total_loss.item()
            print(f"Потеря на валидационном наборе: {val_loss / len(val_loader)}")

# Обучение модели
train_model_(train_loader, val_loader, optimizer, 20)

# Сохранение модели
torch.save(model.state_dict(), "./parses/clip_finetuned.pth")

Эпоха 1/20 - Обучение:   0%|          | 0/112 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'data'